In [1]:
import os, sys, glob, pathlib, random
import numpy as np

from matplotlib import pyplot as plt
from PIL import Image

In [2]:
project_dir = os.path.split(os.getcwd())[0]

# Augmentor

In [3]:
augmentor_dir = os.path.join(project_dir, "3rdParty/Augmentor/")
if augmentor_dir not in sys.path:
    sys.path.append(augmentor_dir)

from Augmentor import Operations as AugOper

# Augmentation

Se aplican de forma aleatoria las siguientes transformaciones:
    
* Distorción elástica
* Zoom
* Rotación

In [4]:
rotate_op = AugOper.RotateStandard(0.7, max_left_rotation = 12, max_right_rotation=12)
zoom_op = AugOper.Zoom(0.7, min_factor=0.8, max_factor=1.2)
distort_op = AugOper.Distort(0.7, grid_width=8, grid_height=8, magnitude=1)
constrast_op = AugOper.RandomContrast(0.7, 0.8, 1.2)

In [5]:
operations = [distort_op, zoom_op, rotate_op]

In [6]:
def augmentation(image: Image, mask: Image):
    r = round(random.uniform(0, 1), 2)
    if r <= constrast_op.probability:
        image = constrast_op.perform_operation([image])[0]

    augmented_imgs = [image, mask]

    for operation in operations:
        r = round(random.uniform(0, 1), 2)
        if r <= operation.probability:
            augmented_imgs = operation.perform_operation(augmented_imgs)

    return augmented_imgs



In [7]:
subjects_imgs_dir = "processed/images"
subjects_mask_dir = "processed/masks"

save_imgs_dir = "augmented/images/"
save_mask_dir = "augmented/masks/"


for subject in os.listdir(subjects_imgs_dir):
    current_subject_img_dir = os.path.join(subjects_imgs_dir, subject)
    current_subject_mask_dir = os.path.join(subjects_mask_dir, subject)
    mask_files = os.listdir(current_subject_mask_dir)

    if not os.path.exists(os.path.join(save_imgs_dir, subject)):
        os.mkdir(os.path.join(save_imgs_dir, subject))
    
    if not os.path.exists(os.path.join(save_mask_dir, subject)):
        os.mkdir(os.path.join(save_mask_dir, subject))

    for filename in os.listdir(current_subject_img_dir):
        if not(filename in mask_files):
            print("There is no a mask for {}".filename)
            continue

        augmented_filename = filename.split(".")[0] + "_AUG.png"

        image = Image.open(os.path.join(current_subject_img_dir, filename))
        mask = Image.open(os.path.join(current_subject_mask_dir, filename))
        image, mask = augmentation(image, mask)

        image.save(os.path.join(save_imgs_dir, subject, augmented_filename))
        mask.save(os.path.join(save_mask_dir, subject, augmented_filename))

